In [1]:
import utils
import params
import os
from tqdm import tqdm
import numpy as np
import xlsxwriter
from collections import defaultdict

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.preprocessing import image

C:\Users\bldr1\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### CHOOSE 1000 TEST QUERIES

In [ ]:
captions = utils.readCaptions(params.VAL_CAPTIONS_PATH)

In [ ]:
captions[0]

In [ ]:
"""N = 1011
captions_idxs = np.arange(captions.__len__())
np.random.shuffle(captions_idxs)"""

In [ ]:
"""captions_dict = {}
for caption_idx in tqdm(captions_idxs[:N]):
    caption = captions[caption_idx]['caption']
    caption_id = captions[caption_idx]['image_id']
    caption_tfidf = np.load( params.TFIDF_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy').tolist()[0]
    caption_rtfidf = np.load( params.REDUCED_TFIDF_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy').tolist()[0]
    caption_w2v = np.load(  params.W2V_DESCRIPTORS_VAL_PATH+str(caption_id)+'.npy' )[0]
    captions_dict[str(caption_id)] = {}
    captions_dict[str(caption_id)]['caption'] = caption
    captions_dict[str(caption_id)]['tfidf'] = caption_tfidf
    captions_dict[str(caption_id)]['r-tfidf'] = caption_rtfidf
    captions_dict[str(caption_id)]['w2v'] = caption_w2v"""

In [ ]:
"""captions_dict.keys().__len__()"""

In [ ]:
"""utils.savePickle(object=captions_dict, PATH='TMP/test_captions')"""

In [2]:
captions_dict = utils.importPickle('TMP/test_captions')

In [3]:
OUTPUT_SHAPE = 128
TFIDF_WEIGHTS = 'TMP/model_tfidf_vlad_pca_dropout_02.h5'
RTFIDF_WEIGHTS = 'TMP/model_reduced_tfidf_vlad_pca_dropout_02.h5'
W2V_WEIGHTS = 'TMP/model_w2v_vlad_pca_dropout_02.h5'

In [4]:
INPUT_SHAPE = 369537
tfidf_model = Sequential()
tfidf_model.add(Dense(input_dim=INPUT_SHAPE, units = 150, kernel_initializer='normal'))
tfidf_model.add(Activation('relu'))
tfidf_model.add(Dropout(0.2))
tfidf_model.add(Dense(units = 130, kernel_initializer='normal'))
tfidf_model.add(Activation('relu'))
tfidf_model.add(Dropout(0.2))
tfidf_model.add(Dense(units = OUTPUT_SHAPE, kernel_initializer='normal'))
tfidf_model.compile(loss='mean_squared_error',optimizer='adam')

tfidf_model.load_weights(TFIDF_WEIGHTS)

In [5]:
INPUT_SHAPE = 91155
rtfidf_model = Sequential()
rtfidf_model.add(Dense(input_dim=INPUT_SHAPE, units = 150, kernel_initializer='normal'))
rtfidf_model.add(Activation('relu'))
rtfidf_model.add(Dropout(0.2))
rtfidf_model.add(Dense(units = 130, kernel_initializer='normal'))
rtfidf_model.add(Activation('relu'))
rtfidf_model.add(Dropout(0.2))
rtfidf_model.add(Dense(units = OUTPUT_SHAPE, kernel_initializer='normal'))
rtfidf_model.compile(loss='mean_squared_error',optimizer='adam')

rtfidf_model.load_weights(RTFIDF_WEIGHTS)

In [6]:
INPUT_SHAPE = 300
w2v_model = Sequential()
w2v_model.add(Dense(input_dim=INPUT_SHAPE, units = 150, kernel_initializer='normal'))
w2v_model.add(Activation('relu'))
w2v_model.add(Dropout(0.2))
w2v_model.add(Dense(units = 130, kernel_initializer='normal'))
w2v_model.add(Activation('relu'))
w2v_model.add(Dropout(0.2))
w2v_model.add(Dense(units = OUTPUT_SHAPE, kernel_initializer='normal'))
w2v_model.compile(loss='mean_squared_error',optimizer='adam')

w2v_model.load_weights(W2V_WEIGHTS)

In [7]:
tree = utils.importPickle(params.VLAD_VAL_INDEX)
order = utils.importPickle(params.VLAD_VAL_ORDER)

In [8]:
"""captions_dict = utils.importPickle('TMP/cnn_captions')"""

"captions_dict = utils.importPickle('TMP/cnn_captions')"

In [9]:
captions_dict['174390']

{'caption': 'A person on a motor bike on a street.',
 'r-tfidf': <1x91155 sparse matrix of type '<class 'numpy.float64'>'
 	with 9 stored elements in Compressed Sparse Row format>,
 'tfidf': <1x369537 sparse matrix of type '<class 'numpy.float64'>'
 	with 9 stored elements in Compressed Sparse Row format>,
 'w2v': array([-2.4500e-02, -1.3288e-01, -7.0820e-02, -1.3780e-02, -7.8200e-03,
        -5.5000e-03,  6.7920e-02,  7.0820e-02, -8.1000e-03, -3.2000e-03,
         5.8680e-02,  6.9800e-02,  1.5888e-01,  5.8780e-02,  8.7180e-02,
         7.4220e-02, -3.3020e-02,  3.0000e-04, -2.2700e-02, -5.9740e-02,
        -1.7806e-01, -2.5700e-02, -3.6620e-02,  1.1048e-01,  7.9740e-02,
        -5.1660e-02, -3.0020e-02,  9.8540e-02,  1.0432e-01, -1.7940e-01,
         5.8220e-02, -1.0228e-01, -5.6840e-02,  2.6180e-02,  6.7600e-03,
        -1.6020e-02,  2.8760e-02,  2.6780e-02,  6.0700e-02, -2.4620e-02,
         2.9280e-02,  7.2400e-03, -3.2160e-02, -5.3420e-02, -6.8480e-02,
        -2.2520e-02,  8.0200

In [10]:
for key in tqdm(captions_dict.keys()):
    tfidf_pred = tfidf_model.predict(captions_dict[key]['tfidf'])
    rtfidf_pred = rtfidf_model.predict(captions_dict[key]['r-tfidf'])
    w2v_pred = w2v_model.predict(captions_dict[key]['w2v'].reshape(-1,300))
      
    tfidf_dist, tfidf_idxs = tree.query(tfidf_pred/np.linalg.norm(tfidf_pred), k=10)
    rtfidf_dist, rtfidf_idxs = tree.query(rtfidf_pred/np.linalg.norm(rtfidf_pred), k=10)
    w2v_dist, w2v_idxs = tree.query(w2v_pred/np.linalg.norm(w2v_pred), k=10)
    
    tfidf_images = [order[i] for i in tfidf_idxs[0]] 
    rtfidf_images = [order[i] for i in rtfidf_idxs[0]]
    w2v_images = [order[i] for i in w2v_idxs[0]]
    
    tfidf_paths = [params.VAL_IMAGES_PATH + os.path.basename(image).split('.')[0][4:]+'.jpg' for image in tfidf_images]
    rtfidf_paths = [params.VAL_IMAGES_PATH + os.path.basename(image).split('.')[0][4:]+'.jpg' for image in rtfidf_images]
    w2v_paths = [params.VAL_IMAGES_PATH + os.path.basename(image).split('.')[0][4:]+'.jpg' for image in w2v_images]
    
    captions_dict[key]['vlad_tfidf_paths'] = tfidf_paths
    captions_dict[key]['vlad_rtfidf_paths'] = rtfidf_paths
    captions_dict[key]['vlad_w2v_paths'] = w2v_paths

100%|██████████| 1000/1000 [01:57<00:00,  8.52it/s]


In [11]:
utils.savePickle(captions_dict,'TMP/vlad_captions')

In [12]:
alphabet = ['C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 
            'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [14]:
for counter in range(10):
    tfidf_workbook = xlsxwriter.Workbook('EXCELS/VLAD/TFIDF_VLAD/tfidf_vlad_{}.xlsx'.format(counter))
    tfidf_worksheet = tfidf_workbook.add_worksheet()
    tfidf_worksheet.set_column('A:A', 10)
    tfidf_worksheet.set_column('B:B', 60)
    tfidf_worksheet.write('A1','POSITION')

    j = 4
    for key in tqdm(list(captions_dict.keys())[counter*100:counter*100+100]):
        caption = captions_dict[key]['caption']
        tfidf_paths = captions_dict[key]['vlad_tfidf_paths']
        #rtfidf_paths = captions_dict[key]['rtfidf_paths']
        #w2v_paths = captions_dict[key]['w2v_paths']



        tfidf_worksheet.write('B{}'.format(j), caption)
        for i, path in enumerate(tfidf_paths):
            img = image.load_img(path, target_size=(224, 224))
            name = 'CROP/'+os.path.basename(path)
            image.save_img(name, img)
            tfidf_worksheet.insert_image('{}{}'.format(alphabet[i*2], j-2), 
                                         name, 
                                         {'x_scale': 0.5, 'y_scale': 0.5})

        j = j+6
    tfidf_workbook.close()

100%|██████████| 100/100 [00:27<00:00,  3.70it/s]


In [15]:
for counter in range(10):
    rtfidf_workbook = xlsxwriter.Workbook('EXCELS/VLAD/R-TFIDF_VLAD/rtfidf_vlad_{}.xlsx'.format(counter))
    rtfidf_worksheet = rtfidf_workbook.add_worksheet()
    rtfidf_worksheet.set_column('A:A', 10)
    rtfidf_worksheet.set_column('B:B', 60)
    rtfidf_worksheet.write('A1','POSITION')

    j = 4
    for key in tqdm(list(captions_dict.keys())[counter*100:counter*100+100]):
        caption = captions_dict[key]['caption']
        rtfidf_paths = captions_dict[key]['vlad_rtfidf_paths']
        #rtfidf_paths = captions_dict[key]['rtfidf_paths']
        #w2v_paths = captions_dict[key]['w2v_paths']



        rtfidf_worksheet.write('B{}'.format(j), caption)
        for i, path in enumerate(rtfidf_paths):
            img = image.load_img(path, target_size=(224, 224))
            name = 'CROP/'+os.path.basename(path)
            image.save_img(name, img)
            rtfidf_worksheet.insert_image('{}{}'.format(alphabet[i*2], j-2), 
                                         name, 
                                         {'x_scale': 0.5, 'y_scale': 0.5})

        j = j+6
    rtfidf_workbook.close()

100%|██████████| 100/100 [00:26<00:00,  3.74it/s]


In [16]:
for counter in range(10):
    w2v_workbook = xlsxwriter.Workbook('EXCELS/VLAD/W2V_VLAD/w2v_vlad_{}.xlsx'.format(counter))
    w2v_worksheet = w2v_workbook.add_worksheet()
    w2v_worksheet.set_column('A:A', 10)
    w2v_worksheet.set_column('B:B', 60)
    w2v_worksheet.write('A1','POSITION')

    j = 4
    for key in tqdm(list(captions_dict.keys())[counter*100:counter*100+100]):
        caption = captions_dict[key]['caption']
        w2v_paths = captions_dict[key]['vlad_w2v_paths']
        #rtfidf_paths = captions_dict[key]['rtfidf_paths']
        #w2v_paths = captions_dict[key]['w2v_paths']



        w2v_worksheet.write('B{}'.format(j), caption)
        for i, path in enumerate(w2v_paths):
            img = image.load_img(path, target_size=(224, 224))
            name = 'CROP/'+os.path.basename(path)
            image.save_img(name, img)
            w2v_worksheet.insert_image('{}{}'.format(alphabet[i*2], j-2), 
                                         name, 
                                         {'x_scale': 0.5, 'y_scale': 0.5})

        j = j+6
    w2v_workbook.close()

100%|██████████| 100/100 [00:27<00:00,  3.70it/s]
